In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
import logistic_regression as lr
import naive_bayes as nb

In [7]:
betas = np.loadtxt('../multi_task_model_optimal_1.5_threshold/beta_children.txt')


In [28]:
train = pd.read_csv('../multi_task_model_optimal_1.5_threshold/train.csv', index_col=(0,1))

In [30]:
train.tail()

intercept  max_CpG_10kb  max_PHRED_10kb  \
subject_id gene_id                                                      
GTEX-XXEK  ENSG00000272715.1        1.0      0.676254        0.377575   
           ENSG00000272994.1        1.0      0.717920        0.405084   
           ENSG00000273173.1        1.0      0.592920        0.359083   
           ENSG00000273230.1        1.0      0.717920        0.360658   
           ENSG00000273487.1        1.0      0.634587        0.567045   

                              max_verPhCons_10kb  max_cHmmTssA_10kb  \
subject_id gene_id                                                    
GTEX-XXEK  ENSG00000272715.1            0.577863           1.010185   
           ENSG00000272994.1            0.577017           1.167185   
           ENSG00000273173.1            0.577017           1.010185   
           ENSG00000273230.1            0.577017           1.034185   
           ENSG00000273487.1            0.956433           1.010185   

                              max_mamPhCons_10kb  max_verPhyloP_10kb  \
subject_id gene_id                                                     
GTEX-XXEK  ENSG00000272715.1            0.560688            0.712058   
           ENSG00000272994.1            0.559807            0.657540   
           ENSG00000273173.1            0.559807            0.684588   
           ENSG00000273230.1            0.559807            0.548736   
           ENSG00000273487.1            0.915923            0.700593   

                              max_GC_10kb  max_EncOCpolIIPVal_10kb  \
subject_id gene_id                                                   
GTEX-XXEK  ENSG00000272715.1     1.233815                 1.184023   
           ENSG00000272994.1     1.233815                 1.184023   
           ENSG00000273173.1     1.156892                 1.184023   
           ENSG00000273230.1     1.400482                 2.184023   
           ENSG00000273487.1     1.015867                 1.184023   

                              max_TFBS_10kb              ...               \
subject_id gene_id                                       ...                
GTEX-XXEK  ENSG00000272715.1       0.372614              ...                
           ENSG00000272994.1       0.372614              ...                
           ENSG00000273173.1       0.372614              ...                
           ENSG00000273230.1       0.658328              ...                
           ENSG00000273487.1       0.389420              ...                

                              E113_general_enhancer  expression  eqtl  \
subject_id gene_id                                                      
GTEX-XXEK  ENSG00000272715.1                    0.0    1.010693   1.0   
           ENSG00000272994.1                    0.0    0.410567   1.0   
           ENSG00000273173.1                    0.0    1.055161   1.0   
           ENSG00000273230.1                    1.0    0.388925   1.0   
           ENSG00000273487.1                    0.0    1.071788   1.0   

                              expr_label  posterior  tissue  shared_label  \
subject_id gene_id                                                          
GTEX-XXEK  ENSG00000272715.1         0.0   0.000265  muscle           0.0   
           ENSG00000272994.1         0.0   0.000284  muscle           0.0   
           ENSG00000273173.1         0.0   0.000312  muscle           0.0   
           ENSG00000273230.1         0.0   0.000352  muscle           0.0   
           ENSG00000273487.1         0.0   0.000284  muscle           0.0   

                              posterior_RIVER  posterior_genome_only  \
subject_id gene_id                                                     
GTEX-XXEK  ENSG00000272715.1         0.004535               0.079955   
           ENSG00000272994.1         0.003449               0.082428   
           ENSG00000273173.1         0.004093               0.086897   
           ENSG00000273230.1         0.005848               0.093242   


In [14]:
g = pd.read_csv("../input/g_train.csv", index_col=(0,1))

In [15]:
g.insert(0, "intercept", 1)

In [16]:
g.head()

intercept  max_CpG_10kb  max_PHRED_10kb  \
subject_id gene_id                                                       
GTEX-N7MS  ENSG00000001561.6           1      0.572087        0.376902   
           ENSG00000003056.3           1      0.592920        0.364608   
           ENSG00000003402.15          1      0.801254        0.388538   
           ENSG00000004534.10          1      0.676254        0.427440   
           ENSG00000004660.10          1      0.592920        0.448680   

                               max_verPhCons_10kb  max_cHmmTssA_10kb  \
subject_id gene_id                                                     
GTEX-N7MS  ENSG00000001561.6             0.578286           1.010185   
           ENSG00000003056.3             0.582093           1.026185   
           ENSG00000003402.15            0.577017           1.687185   
           ENSG00000004534.10            0.602396           1.018185   
           ENSG00000004660.10            0.583785           1.018185   

                               max_mamPhCons_10kb  max_verPhyloP_10kb  \
subject_id gene_id                                                      
GTEX-N7MS  ENSG00000001561.6             0.560247            0.672983   
           ENSG00000003056.3             0.563769            0.676492   
           ENSG00000003402.15            0.559807            0.668350   
           ENSG00000004534.10            0.584898            0.704711   
           ENSG00000004660.10            0.565090            0.733772   

                               max_GC_10kb  max_EncOCpolIIPVal_10kb  \
subject_id gene_id                                                    
GTEX-N7MS  ENSG00000001561.6      1.015867                 1.184023   
           ENSG00000003056.3      1.156892                 1.184023   
           ENSG00000003402.15     1.451764                 1.344023   
           ENSG00000004534.10     1.323559                 1.184023   
           ENSG00000004660.10     1.323559                 1.266523   

                               max_TFBS_10kb          ...            \
subject_id gene_id                                    ...             
GTEX-N7MS  ENSG00000001561.6        0.372614          ...             
           ENSG00000003056.3        0.372614          ...             
           ENSG00000003402.15       0.582698          ...             
           ENSG00000004534.10       0.372614          ...             
           ENSG00000004660.10       0.372614          ...             

                               E107_general_promoter  E107_general_enhancer  \
subject_id gene_id                                                            
GTEX-N7MS  ENSG00000001561.6                       0                      0   
           ENSG00000003056.3                       0                      1   
           ENSG00000003402.15                      1                      0   
           ENSG00000004534.10                      0                      0   
           ENSG00000004660.10                      1                      0   

                               E108_general_promoter  E108_general_enhancer  \
subject_id gene_id                                                            
GTEX-N7MS  ENSG00000001561.6                       0                      0   
           ENSG00000003056.3                       0                      1   
           ENSG00000003402.15                      1                      0   
           ENSG00000004534.10                      0                      0   
           ENSG00000004660.10                      1                      0   

                               E110_general_promoter  E110_general_enhancer  \
subject_id gene_id                                                            
GTEX-N7MS  ENSG00000001561.6                       0                      0   
           ENSG00000003056.3                       0                      1   
           ENSG00000003402.15                      1                      0

In [22]:
x = pd.DataFrame()
x = x.append(list(g.columns))

In [26]:
for i in range(len(betas)):
    x[i+1] = betas[i]

In [ ]:
x[0] == 

In [27]:
x

,0,1,2,3,4,5
0,intercept,-3.740826,-4.851970,-4.719344,-4.985643,-2.957723
1,max_CpG_10kb,0.150164,-0.544695,-0.400061,0.135975,-0.343619
2,max_PHRED_10kb,0.558791,0.113464,0.373385,-0.291462,0.260063
3,max_verPhCons_10kb,0.072577,-0.253842,-0.268562,0.291569,-0.190062
4,max_cHmmTssA_10kb,0.026676,-0.036936,-0.487707,0.163838,0.304285
5,max_mamPhCons_10kb,0.351654,0.798167,0.869009,0.833587,0.650747
6,max_verPhyloP_10kb,0.821609,0.602984,0.626495,0.817473,-0.127381
7,max_GC_10kb,-0.287898,0.134132,0.197272,-0.392608,-0.172263
8,max_EncOCpolIIPVal_10kb,0.284526,0.093145,0.269152,0.155699,-0.133717
9,max_TFBS_10kb,-0.264823,-0.355914,-0.347732,-0.169337,0.157699
